In [ ]:
%run ./../data/load-tokenized-dataset.ipynb
%run ./../doc2vec/_load-d2v-model.ipynb
%matplotlib inline

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorboard.plugins import projector
from sklearn.decomposition import PCA
from collections import Counter

In [ ]:
RANDOM_SEED = 0

In [ ]:
doc_idx = 10
sample_doc = df[proc_doc_col][doc_idx]

inferred_dv = d2v_model.infer_vector(sample_doc)
sims = d2v_model.dv.most_similar([inferred_dv], topn=len(d2v_model.dv))

print('\033[1m', f"Document [{doc_idx}]:", '\033[0m', '\n', f"«{df[doc_col][doc_idx][:1000]}…»", '\n')
for i, label in [(0,            "Most similar"),
                 (1,            "Secondmost similar"), 
                 (len(df) // 2, "Median similar"), 
                 (len(df) - 1,  "Least similar")]:
    print('\033[1m', f"{label} [{sims[i][0]}, {sims[i][1]}]:", '\033[0m', '\n', f"«{df[doc_col][sims[i][0]][:1000]}…»", '\n')

In [ ]:
docs, labels = d2v_model.dv.vectors, df[label_col].values

reduced_docs = PCA(n_components=2, random_state=RANDOM_SEED).fit_transform(docs)

fig = plt.figure(figsize=(12, 12))
ax = fig.gca()
ax.scatter(*reduced_docs[[label for label in labels]].T, alpha=.1, c='#1f77b4')
ax.scatter(*reduced_docs[[not label for label in labels]].T, alpha=.1, c='#d62728')

In [ ]:
dv = d2v_model.dv.vectors
tf.train.Checkpoint(embedding=tf.Variable(dv)).save(f'{D2V_MODEL_DIR}/embedding.ckpt')
with open(f'{D2V_MODEL_DIR}/embedding.meta', 'w', encoding='utf-8') as f:
    [f.write(str(int(label)) + '\n') for label in df[label_col]]

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = 'embedding/.ATTRIBUTES/VARIABLE_VALUE'
embedding.metadata_path = 'embedding.meta'
projector.visualize_embeddings(D2V_MODEL_DIR, config)

# select projector in dropdown menu
%reload_ext tensorboard
%tensorboard --logdir $D2V_MODEL_DIR